## Создание приложения для предсказания стоимости европейского автомобиля по VIN-коду

### 1. Задача

Нам предосталлен текстовый файл с собранными данными о vin-кодах различных стран и производителей. В тексте vin и стоимость расположены с учетом паттерна "\n[VIN:цена]\n". Наша задача, имея только эту информацию, разделить, с помощью регулярных выражений, вин код на части, а затем, обучив модель машинного обучения создать пользовательское приложение. Так как для разных стран производителей существуют разные правила формирования VIN, мы ограничимся только Европой.

In [483]:
import re
import pandas as pd
import numpy as np
import pickle
from flask import Flask, request, render_template_string

import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline

### 2. Подключение к данным

In [451]:
file_path = 'all_models_train.txt'
with open(file_path, 'r') as file:
    text = file.read()

Валидность vin будем проверять по наличию запрещенных символов I, O и Q, длине в 7мь знаков, а также наличию цифр в конце кода. Проверку сразу включим в парсинг имеющихся данных и разобьем на именованные группы для удобного создания датафрейма. Также, в проверку включим ограничение стран Европы, которые указываются в первой части кода STUVWXYZ. 

### 3. Создание датасета и преддобработка

In [452]:
regex = re.compile(r'''
    (?P<country>[STUVWXYZ1-9])
    (?P<manufacturer>[A-HJ-NPR-Z])
    (?P<vehicle>[A-HJ-NPR-Z0-9]{5})
    (?P<check_digit>[0-9X])
    (?P<year>[A-HJ-NPR-Z0-9])
    (?P<plant_code>[A-HJ-NPR-Z0-9])
    (?P<serial_number>[A-HJ-NPR-Z0-9]{6})
    :(?P<price>\d+)
''', re.X)


df = pd.DataFrame([x.groupdict() for x in regex.finditer(text)])

In [453]:
df.head()

,country,manufacturer,vehicle,check_digit,year,plant_code,serial_number,price
0,T,F,BV541,5,7,X,019560,17250
1,V,W,EK73C,2,6,P,046847,4000
2,Z,V,BP8EM,0,D,5,237253,11600
3,X,X,GM4A7,4,E,G,346791,15400
4,T,H,CK262,2,7,2,010506,9000


In [454]:
df.dtypes

country          object
manufacturer     object
vehicle          object
check_digit      object
year             object
plant_code       object
serial_number    object
price            object
dtype: object

In [455]:
df['price'] = df['price'].astype(int)

In [456]:
df.isna().sum()

country          0
manufacturer     0
vehicle          0
check_digit      0
year             0
plant_code       0
serial_number    0
price            0
dtype: int64

In [381]:
df.duplicated().sum()

0

In [382]:
df.shape

(48226, 8)

Получили датафрейм с кодом страны, производителя, атрибутами двигателя, проверочной цыфрой, годом производства, кодом завода, серийным номером и соответсвующей ценой. Пропусков нет, дубликатов тоже. 

### 4. Подготовка данных к моделированию

Категориальные признаки закодируем с помощью label_encoders


In [457]:
ordinal_encoders = {}
column_cat = ['country', 'manufacturer', 'vehicle', 'check_digit', 'year', 'plant_code', 'serial_number']

for column in column_cat:
    ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    df[column] = ordinal_encoder.fit_transform(df[[column]])
    ordinal_encoders[column] = ordinal_encoder

Создадим валидационные и тестовые выборки для чистого эксперимента

In [458]:
y = df['price']

X_train, X_test_valid, y_train, y_test_valid = train_test_split(X, y, test_size=0.3, random_state=33)
X_valid, X_test, y_valid, y_test = train_test_split(X_test_valid, y_test_valid, test_size=0.5, random_state=33)

значения сильно разбросаны по значениям - скалируем параметры

In [397]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### 5. Обучение модели

У нас большая часть данных это категории переведенные в код - лучше всего должен справиться решающий лес

In [398]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=33)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 2577.468970865716


In [386]:
l_model = LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=100)

l_model.fit(X_train, y_train, categorical_feature=all)


y_pred = l_model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print(f'Root Mean Squared Error: {rmse}')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001101 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 600
[LightGBM] [Info] Number of data points in the train set: 33758, number of used features: 7
[LightGBM] [Info] Start training from score 14536.378518
Root Mean Squared Error: 2824.7145561138937


### 6. Подбор гиперпараметров

Попробуем улучшить подбором параметров

In [399]:
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6]
}

random_search = RandomizedSearchCV(
    estimator=rf_model, 
    param_distributions=param_dist, 
    n_iter=20, cv=3, scoring='neg_mean_squared_error',
    verbose=2, random_state=42
)

random_search.fit(X_train, y_train)

best_rf_model = random_search.best_estimator_

# Оценка лучшей модели
y_pred = best_rf_model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print(f"Root Mean Squared Error: {rmse}")
print(f"Best Parameters: {random_search.best_params_}")

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END max_depth=20, min_samples_leaf=4, min_samples_split=5, n_estimators=500; total time=  29.1s
[CV] END max_depth=20, min_samples_leaf=4, min_samples_split=5, n_estimators=500; total time=  28.4s
[CV] END max_depth=20, min_samples_leaf=4, min_samples_split=5, n_estimators=500; total time=  28.5s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=   5.7s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=   5.7s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=   5.6s
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=  24.0s
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=  24.0s
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=  23.9s
[CV] END max_depth=20

Ошибка достаточно высокая, но для примерной оценки можем использовать в качестве отправной точки. В финальных выводах сформируем рекомендации по улучшению модели

### 7. Тестирование модели

In [400]:
y_pred = best_rf_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

2476.3229950377536

При тестировании качество упало незначительно

### 8. Baseline model

In [463]:
dummy_regressor = DummyRegressor(strategy="mean")
dummy_regressor.fit(X_train, y_train)
y_pred = dummy_regressor.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error of baseline model: {rmse}")

Root Mean Squared Error of baseline model: 9512.802640673668


Все же наша модель луче чем простое среднее - использование модели считаем оправданным. Запакуем модель


### 9.Запаковка модели

In [485]:
with open('encoder.pkl', 'wb') as file:
    pickle.dump(encoder, file)
with open('model.pkl', 'wb') as file:
    pickle.dump(best_rf_model, file)

### 10. Создание приложения

Для начала соберем части входного вин-кода и закодируем полученную сточку для передачи в модель, а также проведем скалирование

In [507]:
app = Flask(__name__)

HTML_TEMPLATE = '''
<!doctype html>
<html>
<head><title>VIN Price Predictor</title></head>
<body>
    <h1>VIN Price Prediction</h1>
    <form action="/" method="post">
        <label for="vin">Enter VIN:</label>
        <input type="text" id="vin" name="vin" required>
        <button type="submit">Predict</button>
    </form>
    {% if prediction %}
        <h2>Predicted Price: {{ prediction }}</h2>
    {% endif %}
    {% if error %}
        <h2 style="color: red;">{{ error }}</h2>
    {% endif %}
</body>
</html>
'''

@app.route('/', methods=['GET', 'POST'])
def predict():
    with open('encoder.pkl', 'rb') as file:
        encoder = pickle.load(file)

    with open('model.pkl', 'rb') as file:
        best_rf_model = pickle.load(file)
    
    
    prediction = None
    error = None
    if request.method == 'POST':
        vin = request.form['vin']
 
        if not re.fullmatch(r'[A-HJ-NPR-Z0-9]{17}', vin):
            error = 'Invalid VIN. Please enter a valid 17-character VIN (excluding I, O, Q).'

        elif not re.match(r'[S-ZV-X][A-HJ-NPR-Z0-9]{16}', vin):
            error = 'Valid VIN but not from Europe. Please enter a European VIN.'
        else:
            column_cat = ['country', 'manufacturer', 'vehicle', 'check_digit', 'year', 'plant_code', 'serial_number']
            regex_v = re.compile(r'''
                (?P<country>[STUVWXYZ1-9])
                (?P<manufacturer>[A-HJ-NPR-Z])
                (?P<vehicle>[A-HJ-NPR-Z0-9]{5})
                (?P<check_digit>[0-9X])
                (?P<year>[A-HJ-NPR-Z0-9])
                (?P<plant_code>[A-HJ-NPR-Z0-9])
                (?P<serial_number>[A-HJ-NPR-Z0-9]{6})
            ''', re.X)
            value_vin = pd.DataFrame([x.groupdict() for x in regex_v.finditer(vin)])
            prediction = value_vin
            encoded_values = []
            for column, encoder in ordinal_encoders.items():
                encoded_value = encoder.transform([[value_vin.iloc[0][column_cat.index(column)]]])
                encoded_values.append(encoded_value[0])
                
            encoded_values_stacked = np.hstack(encoded_values)
            final_predict = best_rf_model.predict(encoded_values_stacked.reshape(1, -1))
            prediction = final_predict

    return render_template_string(HTML_TEMPLATE, prediction=prediction, error=error)

if __name__ == '__main__':
    app.run(debug=False)  

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Здесь пока до ума не довел. Есть отработка интерфейса, исключений, проверка валидности, но модель внутри не работает.

### 11. Выводы

В рамках работы удалось собрать датасет из простых текстовых данных о вин-кодах со стоимостями. Удалось построить модель машинного обучения. Модель требует доработки - стоит рассмотреть другие регрессоры и поработать с моделями, которые лучше работают с категориальными признаками, добавив перебор гиперпараметров. Также, сократить ошибку поможет включение более глубокого парсинга сегментов vin, например мощности двигателя итд. Приложение также требует доработки